In [ ]:
############ Auteur: Y.s ###########
############ Fev 2024 ##############
import os
import requests
import pandas as pd
import numpy as np
import datetime
import time
import sys
import gzip

folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_meteo = os.path.join(folder_projet_BD, 'Meteo\CSV')
folder_BD_meteo_region = os.path.join(folder_projet_BD, 'Meteo/region')
folder_BD_conso = os.path.join(folder_projet_BD, 'conso-inf36-region')


In [ ]:
# function to convert 
# Test de la conversion du champs date en année, mois, jour et heure
def extraction_ajout_champs_date(df):
    df_info = pd.to_datetime(df_depa_region['AAAAMMJJHH'].astype(str), format ="%Y%m%d%H").apply(lambda t: pd.Series({
    'date': t.date(),
    'year': t.year,
    'month': t.month,
    'month_n': t.strftime("%B"),
    'day': t.day,
    'day_n': t.strftime("%A"),
    'h': t.hour + 1,
    'mn': t.minute,
    's': t.second,
    }))
    df[df_info.columns] = df_info
    return df

In [ ]:
# Aggrégation des données météo par région :
# ======================================================================================
# Nous gardons que les champs suivantes :
# NUM_POSTE   	 numéro Météo-France du poste sur 8 chiffres ==> permet d'identier 
# NOM_USUEL   	 nom usuel du poste
# LAT         	 latitude, négative au sud (en degrés et millionièmes de degré)
# LON         	 longitude, négative à l’ouest de GREENWICH (en degrés et millionièmes de degré)
# ALTI        	 altitude du pied de l'abri ou du pluviomètre si pas d'abri (en m)
# AAAAMMJJHH  	 date de la mesure (année mois jour heure)
# FF          	 force du vent moyenné sur 10 mn, mesurée à 10 m (en m/s et 1/10)
# T           	 température sous abri instantanée (en °C et 1/10)
# U           	 humidité relative (en %)
# ======================================================================================
# Pour 
    # DIF         	  rayonnement diffus horaire en heure UTC (en J/cm2) 
    # DIR         	  rayonnement direct  horaire en heure UTC (en J/cm2)
# Vu qu'ils contiennent beacoup de données manquantes, nous utilsons une autre base avec le rayonnment global à l'échelle d'une région 

# Dans une région, il y plusieurs postes : 
# on remplace les mesurs de tous ces postes par les moments statistiques
    # d'ordre 1  : moyenne, 
    # d'ordre 2 : écrat-type, 
    # d'ordre 3 : asymétrie (skewness), 
    # d'ordre 4 : l'applatissement (kurtosis )
# Fonction pour calculer les moments 
from scipy.stats import skew
from scipy.stats import kurtosis

# Les fonctions à appliquer 
# 25th Percentile
def q25(x):
    return x.quantile(0.25)
# 50th Percentile
def q50(x):
    return x.quantile(0.5)

# 90th Percentile
def q75(x):
    return x.quantile(0.75)
# Suite à un problème avec skew et kurtosis (trop de valeure nan)
#aggregation = {"T":[("moyenne",'mean'), ('STD', 'std'), ('skew', lambda x : skew(x)),('kurtosis', lambda x : kurtosis(x)) ],
#               "FF":[("moyenne",'mean'), ('STD', 'std'), ('skew', lambda x : skew(x)),('kurtosis', lambda x : kurtosis(x)) ],
#               "U":[("moyenne",'mean'), ('STD', 'std'), ('skew', lambda x : skew(x)),('kurtosis', lambda x : kurtosis(x)) ],
#               }
# Je remplace par les quartiles
aggregation = {"T":[("moyenne",'mean'), ('STD', 'std'), ('q25', lambda x : q25(x)),('q50', lambda x : q50(x)), ('q75', lambda x : q75(x)) ],
               "FF":[("moyenne",'mean'), ('STD', 'std'), ('q25', lambda x : q25(x)),('q50', lambda x : q50(x)), ('q75', lambda x : q75(x)) ],
               "U":[("moyenne",'mean'), ('STD', 'std'), ('q25', lambda x : q25(x)),('q50', lambda x : q50(x)), ('q75', lambda x : q75(x)) ],
               }
# Téléchargement de la BD avec région et département
filename = 'departements-region.csv'
file = os.path.join(folder_BD_meteo, filename)
df_depa_region = pd.read_csv(file)
# JE remplace les espace par un '-' dans les noms des régions
df_depa_region['region_name'] = df_depa_region['region_name'].apply(lambda s : s.replace(" ", '-'))

Liste_region = df_depa_region['region_name'].unique().tolist()

# df_depa_region['region_name'].unique() # pour vérifer
# JE supprime les régions  ['Guadeloupe', 'Martinique', 'Guyane','La-Réunion', 'Mayotte', 'Corse']

for elem in ['Guadeloupe', 'Martinique', 'Guyane','La-Réunion', 'Mayotte', 'Corse']:
    Liste_region.remove(elem)

# Construction d'un dictionnaire de clés région et de valeurs la liste des nuémros de départements de chaque région 
dictionnaire__departement_region = {}

for reg in Liste_region:
    dictionnaire__departement_region[reg] = df_depa_region.loc[df_depa_region['region_name']==reg, 'num_dep'].tolist()

# Construction de la nouvelle base avec les caractéristiques décrites précédement
# Un fichir csv par région pour 2023-2024
template_start= 'H_' # début du nom du fichier
template_end_2020='_previous-' + str(2020) + '-' + str(2023) + '.csv'
template_end_2024='_latest-' + str(2024) + '-' + str(2025) + '.csv'
template_end_reg='-' + str(2023) + '-' + str(2024) + '.csv'
columns_to_drop =  ['NUM_POSTE','NOM_USUEL','LAT','LON','ALTI','RR1','QRR1','DRR1',
                    'QDRR1','QFF','DD','QDD','FXY','QFXY','DXY','QDXY','HXY','QHXY',
                    'FXI','QFXI','DXI','QDXI','HXI','QHXI','FF2','QFF2','DD2','QDD2',
                    'FXI2','QFXI2','DXI2','QDXI2','HXI2','QHXI2','FXI3S','QFXI3S','DXI3S',
                    'QDXI3S','HFXI3S','QHFXI3S','QT','TD','QTD','TN','QTN','HTN','QHTN',
                    'TX','QTX','HTX','QHTX','DG','QDG','T10','QT10','T20','QT20','T50',
                    'QT50','T100','QT100','TNSOL','QTNSOL','TN50','QTN50','TCHAUSSEE',
                    'QTCHAUSSEE','DHUMEC','QDHUMEC','QU','UN','QUN','HUN','QHUN','UX',
                    'QUX','HUX','QHUX','DHUMI40','QDHUMI40','DHUMI80','QDHUMI80','TSV','QTSV',
                    'PMER','QPMER','PSTAT','QPSTAT','PMERMIN','QPMERMIN','GEOP','QGEOP','N',
                    'QN','NBAS','QNBAS','CL','QCL','CM','QCM','CH','QCH','N1','QN1','C1','QC1',
                    'B1','QB1','N2','QN2','C2','QC2','B2','QB2','N3','QN3','C3','QC3','B3',
                    'QB3','N4','QN4','C4','QC4','B4','QB4','VV','QVV','DVV200','QDVV200',
                    'WW','QWW','W1','QW1','W2','QW2','SOL','QSOL','SOLNG','QSOLNG','TMER',
                    'QTMER','VVMER','QVVMER','ETATMER','QETATMER','DIRHOULE','QDIRHOULE',
                    'HVAGUE','QHVAGUE','PVAGUE','QPVAGUE','HNEIGEF','QHNEIGEF','NEIGETOT',
                    'QNEIGETOT','TSNEIGE','QTSNEIGE','TUBENEIGE','QTUBENEIGE','HNEIGEFI3',
                    'QHNEIGEFI3','HNEIGEFI1','QHNEIGEFI1','ESNEIGE','QESNEIGE','CHARGENEIGE',
                    'QCHARGENEIGE','GLO','QGLO','GLO2','QGLO2','DIR','QDIR','DIR2','QDIR2',
                    'DIF','QDIF','DIF2','QDIF2','UV','QUV','UV2','QUV2','UV_INDICE','QUV_INDICE',
                    'INFRAR','QINFRAR','INFRAR2','QINFRAR2','INS','QINS','INS2','QINS2','TLAGON',
                    'QTLAGON','TVEGETAUX','QTVEGETAUX','ECOULEMENT','QECOULEMENT']
columns_to_keep = ['AAAAMMJJHH','T', 'FF', 'U']
for reg, list_dep in dictionnaire__departement_region.items():
    print("... Nous traitons la région", reg)
    premier_departement = True
    for dep in list_dep:
        # #Lecture de la dataframe avec des données 2020-2023
        filename  = f"{template_start}{dep}{template_end_2020}" 
        file = os.path.join(folder_BD_meteo, filename)
        
        df =  pd.read_csv(file, sep=';')
        # suppression des colonnes
        df_2023_dep =df[columns_to_keep]

        # On ne garde que les lignes de 2023
        df_2023_dep = df_2023_dep.loc[df_2023_dep['AAAAMMJJHH']>=2023010100]

    # Lecture des fichiers 2024-2025
        filename  = f"{template_start}{dep}{template_end_2024}" 
        file = os.path.join(folder_BD_meteo, filename)
        df_2024_dep =  pd.read_csv(file, sep=';')
        # chargement 
        df =  pd.read_csv(file, sep=';')
        # suppression des colonnes
        df_2024_dep =df[columns_to_keep]

        # On ne garde que les lignes de 2024
        df_2024_dep = df_2024_dep.loc[df_2024_dep['AAAAMMJJHH']<2025010100]

      

        # Union des deux df 2023 et 2024  avec la df de la région
        if premier_departement == True:
            df_2023_2024 = pd.concat([df_2023_dep, df_2024_dep], ignore_index=True)
            premier_departement = False 
        else:
            df_2023_2024 = pd.concat([df_2023_2024,df_2023_dep, df_2024_dep], ignore_index=True)
    # Maintenant on a un df de la région avec les champs souhaités
    # On applique les stats 
    df_stat_2023_2024 = df_2023_2024.groupby('AAAAMMJJHH').agg(aggregation)
    df_stat_2023_2024.columns = df_stat_2023_2024.columns.map('_'.join)
    df_stat_2023_2024.reset_index(inplace=True) # reindexation

    # Extraction et ajout des champs année, mois, jours, heure
    df_info = pd.to_datetime(df_stat_2023_2024['AAAAMMJJHH'].astype(str), 
                                 format ="%Y%m%d%H").apply(lambda t: pd.Series({'date': t.date(),
                                                                                'year': t.year,
                                                                                'month': t.month,
                                                                                'month_n': t.strftime("%B"),
                                                                                'day': t.day,
                                                                                'day_n': t.strftime("%A"),
                                                                                'h': t.hour + 1,
                                                                                'mn': t.minute,
                                                                                's': t.second,
                                                                                }))
    df_stat_2023_2024[df_info.columns] = df_info

    # On sauvegarde dans un CSV : nom   = H_region-2023-2024.csv" 
    filename = f"{template_start}{reg}{template_end_reg}" 
    file = os.path.join(folder_BD_meteo_region, filename)
    df_stat_2023_2024.to_csv(file)



In [ ]:
# Concaténation des deux bases de données conso + météo 
# Pour la consommation 
template_start_in_meteo= 'H_' # début du nom du fichier meteo 
template_start_in_conso = 'conso-inf36-' # début du nom du fichier conso  
template_end_in='-' + str(2023) + '-' + str(2024) + '.csv'

# Pour le fichier csv d'unification des BD
template_start_out= 'conso-inf36-meteo-' # début du nom du fichier conso-meteo
template_end_out='-' + str(2023) + '-' + str(2024) + '.csv'

# Gestion des repertoires 
folder_projet_BD = r"D:\MesDocuments\Formation\DataScientist_PSL\Projet\BD"
folder_BD_meteo = os.path.join(folder_projet_BD, 'Meteo\CSV')
folder_BD_meteo_region = os.path.join(folder_projet_BD, 'Meteo/region')
folder_BD_conso_region = os.path.join(folder_projet_BD, 'conso-inf36-region')
folder_BD_conso_meteo = os.path.join(folder_projet_BD, 'conso-inf36-meteo-region')

if not os.path.isdir(folder_BD_conso_meteo):
    os.mkdir(folder_BD_conso_meteo)
for reg in Liste_region[2:]:
#def data_conso_meteo_fusion (reg):
    print("... Nous traitons la région", reg)
    # Lecture de la base meteo de la région 
    filename = f"{template_start_in_meteo}{reg}{template_end_in}" 
    file = os.path.join(folder_BD_meteo_region, filename)
    # lecture de la BD meteo de la région 
    df_meteo_region = pd.read_csv(file)
    df_meteo_region.drop(columns = ['Unnamed: 0'], axis=1, inplace=True)
    
    
    # Lecture de la base meteo de la région 
    filename = f"{template_start_in_conso}{reg}{template_end_in}" 
    file = os.path.join(folder_BD_conso_region, filename)
    # lecture de la BD meteo de la région 
    df_conso_region = pd.read_csv(file, sep = ';')
  
    print("... Extraction", reg)
# # Extraction et ajout des champs année, mois, jours, heure pour la base conso
    df_info = pd.to_datetime(df_conso_region['Horodate'],utc=True).apply(lambda t: pd.Series({
        'date': t.date(),
        'year': t.year,
        'month': t.month,
        #'month_n': t.strftime("%B"),
        'day': t.day,
        #'day_n': t.strftime("%A"),
        'h': t.hour + 1,
        'mn': t.minute,
        #'s': t.second,
    }))

    df_conso_region[df_info.columns] = df_info
    # Ajout d'une colonne 'AAAAMMJJHH' qui va nous servir comme colonne commune pour unifier (merge) les deux bases
    df_conso_region['AAAAMMJJHH'] = df_info.h + df_info.day*100+ df_info.month*10000 + df_info.year*1000000

    # Suppression des colonnes pour ne pas avoir de doublons
    df_conso_region.drop(columns=['year', 'month', 'day', 'h', 'date', 'Horodate'], inplace=True)
    #df_meteo_region.drop(columns=['mn','s', 'day_n','month_n'], inplace=True)
    df_meteo_region.drop(columns=['mn','s'], inplace=True)

    print("... Fusion", reg)
    # Fusion des deux bases
    df_fusion = df_conso_region.merge(right = df_meteo_region, on ="AAAAMMJJHH", how = 'left')
    print("... Enregistrement dans un csv", reg)
    # Enregistrement dans des csv
    filename = f"{template_start_out}{reg}{template_end_out}" 
    file = os.path.join(folder_BD_conso_meteo, filename)
    # lecture de la BD meteo de la région 
    df_fusion.to_csv(file)


